In [1]:
import tensorflow as tf
from tensorflow import keras
import mlflow
import numpy as np
import io
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from utils import *

In [2]:
def custom_loss(fk_weight, huber_delta):
    def loss_fn(y_true, y_pred):
        # Huber loss for joint angles
        angle_loss = tf.keras.losses.Huber(delta=huber_delta)(y_true, y_pred)
        
        # Forward kinematics loss (using Huber loss)
        fk_true = forward_kinematics_tf(y_true)
        fk_pred = forward_kinematics_tf(y_pred)
        fk_loss = tf.keras.losses.Huber(delta=huber_delta)(fk_true, fk_pred)
        
        # Combine losses
        total_loss = angle_loss + fk_weight * fk_loss
        return total_loss
    return loss_fn

def create_model(config):
    regularizer = None
    if config['regularization'] == 'l1':
        regularizer = keras.regularizers.l1(config['reg_rate'])
    elif config['regularization'] == 'l2':
        regularizer = keras.regularizers.l2(config['reg_rate'])
    elif config['regularization'] == 'l1_l2':
        regularizer = keras.regularizers.l1_l2(l1=config['reg_rate'], l2=config['reg_rate'])

    model = keras.Sequential([
        keras.layers.Dense(128, input_shape=(3,), kernel_regularizer=regularizer),
        keras.layers.Activation('relu'),
        keras.layers.Dropout(config['dropout_rate']),
        
        keras.layers.Dense(256, kernel_regularizer=regularizer),
        keras.layers.Activation('relu'),
        keras.layers.Dropout(config['dropout_rate']),
        
        keras.layers.Dense(256, kernel_regularizer=regularizer),
        keras.layers.Activation('relu'),
        keras.layers.Dropout(config['dropout_rate']),
        
        keras.layers.Dense(128, kernel_regularizer=regularizer),
        keras.layers.Activation('relu'),
        keras.layers.Dropout(config['dropout_rate']),
        
        keras.layers.Dense(3, activation='tanh')
    ])
    
    model.compile(optimizer='adam', loss=custom_loss(config['fk_weight'], config['huber_delta']))
    return model

def run_single_experiment(config):
    (train_inputs, train_outputs), (test_inputs, test_outputs), input_mean, input_std = load_and_preprocess_data()
    
    valid_inputs, test_inputs, valid_outputs, test_outputs = train_test_split(
        test_inputs, test_outputs, test_size=0.5, random_state=42
    )
    
    mlflow.set_experiment(config['experiment_name'])
    
    with mlflow.start_run(run_name=config['model_name']):
        mlflow.log_params(config)
        
        steps_per_epoch = len(train_inputs) // config['batch_size']
        total_steps = steps_per_epoch * config['epochs']
        warmup_steps = int(0.1 * total_steps)
        
        model = create_model(config)
        
        model_summary = io.StringIO()
        model.summary(print_fn=lambda x: model_summary.write(x + '\n'))
        mlflow.log_text(model_summary.getvalue(), "model_summary.txt")
        
        early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        lr_scheduler = CosineDecayWithWarmupCallback(
            config['initial_learning_rate'],
            warmup_steps,
            total_steps
        )
        verbose_logging = VerboseLoggingCallback()
        lr_logger = LearningRateLogger()
        
        callbacks = [lr_scheduler, verbose_logging, lr_logger, early_stopping]
        
        callback_names = [callback.__class__.__name__ for callback in callbacks]
        mlflow.log_param("callbacks", ", ".join(callback_names))
        
        start_time = time.time()
        history = model.fit(
            train_inputs, train_outputs,
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            validation_data=(valid_inputs, valid_outputs),
            callbacks=callbacks
        )
        training_time = time.time() - start_time
        
        for epoch, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
            mlflow.log_metric("train_loss", loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
        
        mlflow.log_metric("training_time", training_time)
        
        mlflow.tensorflow.log_model(model, "model")
        
        mlflow.log_text(verbose_logging.get_output(), "training_output.txt")
        
        errors, true_xyz, predicted_xyz = evaluate_model(model, test_inputs, test_outputs, input_mean, input_std)
        
        mlflow.log_metric("mean_error", np.mean(errors))
        mlflow.log_metric("median_error", np.median(errors))
        mlflow.log_metric("90th_percentile_error", np.percentile(errors, 90))
        mlflow.log_metric("max_error", np.max(errors))
        
        true_vs_pred_plot_path = f"./Figures/Regularization/{config['model_name']}_true_vs_predicted.png"
        plot_true_vs_predicted(true_xyz, predicted_xyz, f"{config['model_name']} Model: True vs Predicted", save_path=true_vs_pred_plot_path)
        mlflow.log_artifact(true_vs_pred_plot_path)
        
        error_dist_plot_path = f"./Figures/Regularization/{config['model_name']}_error_distribution.png"
        plot_error_distribution(errors, f"{config['model_name']} Model: Error Distribution", save_path=error_dist_plot_path)
        mlflow.log_artifact(error_dist_plot_path)
        
        print(f"\n{config['model_name']} Model:")
        print(f"Mean Error: {np.mean(errors):.4f}")
        print(f"Median Error: {np.median(errors):.4f}")
        print(f"90th Percentile Error: {np.percentile(errors, 90):.4f}")
        print(f"Max Error: {np.max(errors):.4f}")
        print(f"Training Time: {training_time:.2f} seconds")
        
        return {
            'model': model,
            'history': history,
            'errors': errors,
            'true_xyz': true_xyz,
            'predicted_xyz': predicted_xyz,
            'training_time': training_time
        }

In [3]:
# Create configurations
configs = []
regularization_types = [None, 'l1', 'l2', 'l1_l2']
dropout_rates = [0.0, 0.1, 0.3]
huber_deltas = [0.01, 0.02, 0.05]
learning_rates = [0.001, 0.01, 0.05]
reg_rates = [0.0001, 0.001, 0.01]  # New list for regularization rates

for reg_type in regularization_types:
    for dropout_rate in dropout_rates:
        for huber_delta in huber_deltas:
            for learning_rate in learning_rates:
                if reg_type is None:
                    config = {
                        "model_name": f"Model_Reg_None_Dropout_{dropout_rate}_HuberDelta_{huber_delta}_LR_{learning_rate}",
                        "fk_weight": 10,
                        "epochs": 50,
                        "initial_learning_rate": learning_rate,
                        "batch_size": 65536,
                        "experiment_name": "Inverse Kinematics Regularization",
                        "regularization": None,
                        "reg_rate": 0.0,
                        "dropout_rate": dropout_rate,
                        "huber_delta": huber_delta
                    }
                    configs.append(config)
                else:
                    for reg_rate in reg_rates:
                        config = {
                            "model_name": f"Model_Reg_{reg_type}_Rate_{reg_rate}_Dropout_{dropout_rate}_HuberDelta_{huber_delta}_LR_{learning_rate}",
                            "fk_weight": 10,
                            "epochs": 50,
                            "initial_learning_rate": learning_rate,
                            "batch_size": 65536,
                            "experiment_name": "Inverse Kinematics Regularization",
                            "regularization": reg_type,
                            "reg_rate": reg_rate,
                            "dropout_rate": dropout_rate,
                            "huber_delta": huber_delta
                        }
                        configs.append(config)

# Print the total number of configurations
print(f"Total number of configurations: {len(configs)}")

Total number of configurations: 270


In [4]:
# Run experiments
for config in configs:
    run_single_experiment(config)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 317, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 410, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 1341, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/store/tracking/file_store.py", line 1334, in _read_helper
    result = read_yaml(root, file_name)
  File "/usr/local/lib/python3.8/dist-packages/mlflow/utils/file_utils.py", line 309, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingConfigException: Yaml file '/tf/workdir/mlruns/mlruns/meta.yaml' does not

Epoch 1/50
138/138 [==============================] - 3s 12ms/step - loss: 0.0435 - val_loss: 0.0178
Epoch 2/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0122 - val_loss: 0.0093
Epoch 3/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0086 - val_loss: 0.0084
Epoch 4/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0075 - val_loss: 0.0070
Epoch 5/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0070 - val_loss: 0.0065
Epoch 6/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0063 - val_loss: 0.0061
Epoch 7/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0059 - val_loss: 0.0057
Epoch 8/50
138/138 [==============================] - 1s 11ms/step - loss: 0.0055 - val_loss: 0.0053
Epoch 9/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0048 - val_loss: 0.0047
Epoch 10/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0038 - val_lo

138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 40/50
138/138 [==============================] - 1s 11ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 41/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 42/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 43/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 44/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 45/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 46/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 47/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 48/50
138/138 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss:

2024/08/20 18:54:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpvatmtooy/model/data/model/assets


...

Epoch 1/50
138/138 [==============================] - 3s 15ms/step - loss: 20.8841 - val_loss: 1.7941
Epoch 2/50
138/138 [==============================] - 2s 14ms/step - loss: 2.5166 - val_loss: 3.3107
Epoch 3/50
138/138 [==============================] - 2s 14ms/step - loss: 4.0441 - val_loss: 4.7973
Epoch 4/50
138/138 [==============================] - 2s 14ms/step - loss: 5.5673 - val_loss: 6.3036
Epoch 5/50
138/138 [==============================] - 2s 14ms/step - loss: 7.0995 - val_loss: 7.8544
Epoch 6/50
138/138 [==============================] - 2s 14ms/step - loss: 7.8297 - val_loss: 7.6800


2024/08/20 22:52:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmp84xt_ypf/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp84xt_ypf/model/data/model/assets
2024/08/20 22:52:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp84xt_ypf/model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2024/08/20 22:52:13 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


8/8 [==============================] - 0s 3ms/step

Model_Reg_l1_l2_Rate_0.01_Dropout_0.3_HuberDelta_0.05_LR_0.05 Model:
Mean Error: 1.7627
Median Error: 1.6701
90th Percentile Error: 3.1329
Max Error: 3.6632
Training Time: 13.10 seconds
